<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/SorterAnalyticsSpecialist_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Dosya yolunu belirtiyoruz
file_path = "DataCoSupplyChainDataset.csv"

# Veriyi yükle (encoding='latin-1' ekleyerek hata giderildi)
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "shashwatwork/dataco-smart-supply-chain-for-big-data-analysis",
  file_path,
  pandas_kwargs={"encoding": "latin-1"}
)

# İlk 5 kaydı kontrol et
print("Veri başarıyla yüklendi!")
df.head()

In [ ]:
import pandas as pd
import numpy as np

# 1. Veriyi yükle (Daha önce konuştuğumuz encoding ayarı ile)
file_path = "DataCoSupplyChainDataset.csv"
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "shashwatwork/dataco-smart-supply-chain-for-big-data-analysis",
    file_path,
    pandas_kwargs={"encoding": "latin-1"}
)

# 2. Tarih kolonlarını datetime formatına çevir
df['order_date'] = pd.to_datetime(df['order date (DateOrders)'])
df['shipping_date'] = pd.to_datetime(df['shipping date (DateOrders)'])

# 3. İşlem Süresi (Processing Time) Hesaplama
# Siparişin verilmesi ile kargoya verilmesi arasında geçen gerçek gün sayısı
df['actual_processing_days'] = (df['shipping_date'] - df['order_date']).dt.days

# 4. Gecikme Analizi (Scheduled vs Real)
# Planlanan süreden daha uzun süren işlemleri işaretle
df['delay_amount'] = df['Days for shipping (real)'] - df['Days for shipment (scheduled)']
df['is_late'] = df['delay_amount'] > 0

# 5. Sorter Verimliliği İçin Darboğaz Analizi
# Hangi kategorilerde veya hangi günlerde yığılma var?
bottleneck_analysis = df.groupby('Category Name').agg({
    'actual_processing_days': 'mean',
    'is_late': 'sum',
    'Order Id': 'count'
}).rename(columns={'Order Id': 'Total_Orders'}).sort_values(by='actual_processing_days', ascending=False)

print("--- İlk 5 Satır ---")
print(df[['order_date', 'shipping_date', 'actual_processing_days', 'Days for shipment (scheduled)', 'is_late']].head())

print("\n--- Kategori Bazlı Darboğaz Analizi ---")
print(bottleneck_analysis.head(10))

# 6. Basit Hacim Simülasyonu (Örnek)
# Günlük sipariş hacmini hesaplayalım
daily_orders = df.groupby(df['order_date'].dt.date).size()
current_max_capacity = daily_orders.mean() * 1.2  # Mevcut ortalamanın %20 fazlasını kapasite sayalım

print(f"\nGünlük Ortalama Sipariş: {daily_orders.mean():.2f}")
print(f"Varsayılan Sorter Kapasitesi: {current_max_capacity:.2f}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Günlük Sipariş Hacmini Hesapla
daily_demand = df.groupby(df['order_date'].dt.date).size().reset_index()
daily_demand.columns = ['date', 'order_count']

# 2. Sorter Simülasyon Fonksiyonu
def run_sorter_simulation(demand_data, capacity_increase_factor=1.0, demand_spike_factor=1.2):
    """
    capacity_increase_factor: Mevcut kapasiteyi ne kadar artırdığımız (1.0 = değişim yok)
    demand_spike_factor: Sipariş hacmindeki artış (%20 artış için 1.2)
    """
    # Mevcut ortalama kapasiteyi baz alalım (Günlük ortalama siparişin %110'u kapasitemiz olsun)
    base_capacity = demand_data['order_count'].mean() * 1.1 * capacity_increase_factor

    backlog = 0  # Biriken/Yetişmeyen sipariş
    results = []

    for orders in demand_data['order_count']:
        current_demand = orders * demand_spike_factor + backlog

        if current_demand > base_capacity:
            shipped = base_capacity
            backlog = current_demand - base_capacity
        else:
            shipped = current_demand
            backlog = 0

        results.append({
            'Demand': current_demand,
            'Shipped': shipped,
            'Backlog': backlog
        })

    return pd.DataFrame(results), base_capacity

# 3. Simülasyonu Çalıştır (%20 Talep Artışı Senaryosu)
sim_df, daily_cap = run_sorter_simulation(daily_demand, demand_spike_factor=1.2)

# 4. Sonuçları Görselleştir
plt.figure(figsize=(12, 6))
plt.plot(sim_df['Backlog'], color='red', label='Biriken Sipariş (Backlog)')
plt.axhline(y=daily_cap, color='green', linestyle='--', label='Günlük Sorter Kapasitesi')
plt.title('Sipariş Hacmi %20 Artarsa Depo Backlog Durumu')
plt.xlabel('Günler')
plt.ylabel('Sipariş Sayısı')
plt.legend()
plt.show()

# 5. Raporlama
print(f"Günlük Sorter Kapasitesi: {daily_cap:.0f} birim")
print(f"Simülasyon Sonu Toplam Biriken (Yetişmeyen) Sipariş: {sim_df['Backlog'].sum():.0f}")
print(f"Ortalama Günlük Gecikme: {sim_df['Backlog'].mean():.2f} sipariş")

In [ ]:


# Dashboard için ana metrikleri içeren tabloyu oluşturalım
dashboard_data = df.groupby(['Category Name', 'Order Region', 'is_late']).agg({
    'Order Id': 'count',
    'Sales': 'sum',
    'actual_processing_days': 'mean',
    'Benefit per order': 'sum'
}).reset_index()

# Sütun isimlerini dashboard dostu yapalım
dashboard_data.columns = ['Kategori', 'Bolge', 'Gecikme_Durumu', 'Siparis_Sayisi', 'Toplam_Satis', 'Ort_Islem_Suresi', 'Toplam_Kar']

# Dosyayı Excel veya CSV olarak kaydet (Power BI/Tableau'ya yüklemek için)
dashboard_data.to_csv("dashboard_hazir_veri.csv", index=False)
print("Dashboard verisi 'dashboard_hazir_veri.csv' olarak kaydedildi!")

In [ ]:
import plotly.express as px

# 1. Gecikme Oranları (Kategorik)
fig1 = px.bar(dashboard_data, x='Kategori', y='Siparis_Sayisi', color='Gecikme_Durumu',
             title="Kategori Bazlı Gecikme Dağılımı", barmode='group')
fig1.show()

# 2. Bölgesel Satış ve Lojistik Performans
fig2 = px.scatter(dashboard_data, x='Ort_Islem_Suresi', y='Toplam_Satis',
                 size='Siparis_Sayisi', color='Bolge', hover_name='Kategori',
                 title="İşlem Süresi vs Satış Hacmi (Bölgesel)")
fig2.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# 1. Özellik Seçimi (Features) ve Hedef Değişken (Target)
# Tahmin yaparken 'sipariş verildikten sonra oluşan' verileri (gerçek teslimat günü gibi) kullanmamalıyız.
features = [
    'Type', 'Days for shipment (scheduled)', 'Sales per customer',
    'Category Name', 'Order City', 'Order Region', 'Shipping Mode'
]
X = df[features].copy()
y = df['Late_delivery_risk'] # 1: Gecikme riski var, 0: Yok

# 2. Kategorik Verileri Sayısallaştırma (Encoding)
le = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = le.fit_transform(X[col])

# 3. Eğitim ve Test Setine Ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Model Kurulumu (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. Tahmin ve Değerlendirme
y_pred = model.predict(X_test)

print("--- Model Performansı ---")
print(classification_report(y_test, y_pred))

# 6. Önemli Özellikler (Hangi faktör gecikmeyi en çok etkiliyor?)
feature_importance = pd.DataFrame({'Feature': features, 'Importance': model.feature_importances_})
print("\n--- Gecikmeyi Etkileyen En Önemli Faktörler ---")
print(feature_importance.sort_values(by='Importance', ascending=False))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# XGBoost Modelini Tanımla
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=7, random_state=42)

# Modeli Eğit
xgb_model.fit(X_train, y_train)

# Tahmin Yap
y_pred_xgb = xgb_model.predict(X_test)

print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print("\n--- Yeni Model Performansı ---")
print(classification_report(y_test, y_pred_xgb))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('XGBoost Hata Matrisi (Confusion Matrix)')
plt.ylabel('Gerçek Durum')
plt.xlabel('Modelin Tahmini')
plt.show()

In [ ]:
readme_content = """# # 🚚 Veriye Dayalı Tedarik Zinciri ve Lojistik Optimizasyonu

Bu proje, **DataCo Global** şirketinin tedarik zinciri verilerini kullanarak; depo operasyonel verimliliğini artırmak, teslimat gecikmelerini önceden tahmin etmek ve sorter (ayrıştırıcı) sistemlerindeki darboğazları simüle etmek amacıyla geliştirilmiştir.

## 📌 Proje Özeti
Yüksek hacimli lojistik verileri analiz edilerek, teslimat süreçlerindeki aksaklıkların kök nedenleri saptanmış ve **Makine Öğrenmesi (XGBoost)** ile gecikme riski taşıyan siparişler için bir erken uyarı sistemi modellenmiştir.

## 🛠️ Kullanılan Teknolojiler
* **Veri Analizi:** Python, Pandas, Numpy
* **Makine Öğrenmesi:** XGBoost, Random Forest, Scikit-learn
* **Görselleştirme:** Matplotlib, Seaborn, Plotly (Dashboard hazırlığı)
* **Simülasyon:** Python tabanlı kapasite ve backlog (birikme) modelleme

## 🚀 Öne Çıkan Analizler

### 1. Depo ve Sorter Simülasyonu
* Sipariş hacminde gerçekleşecek **%20'lik bir artışın**, mevcut sorter kapasitesiyle yönetilemeyeceği ve operasyonel birikmeye (backlog) yol açacağı kanıtlanmıştır.
* Günlük ortalama kapasite sınırları belirlenerek, yoğun kampanya dönemleri için kaynak planlaması önerileri sunulmuştur.

### 2. Teslimat Gecikmesi Tahminleme (ML)
* **Model:** XGBoost Classifier
* **Performans:** %82 Precision (Gecikme tahminlerinde yüksek doğruluk).
* **Kritik Bulgular:** Gecikmelerin en büyük nedeninin ürün türünden ziyade **Order City (%31)** ve **Sales per customer (%30)** olduğu saptanmıştır.

## 📊 Dashboard ve Görselleştirme
Proje kapsamında hazırlanan görselleştirmeler şunları içerir:
* **Hata Matrisi (Confusion Matrix):** Modelin tahmin başarısının görsel analizi.
* **Feature Importance:** Lojistik performansını etkileyen en kritik değişkenlerin sıralaması.
* **Backlog Trendi:** Kapasite aşımı durumunda siparişlerin günlere göre birikme grafiği.

## 📂 Dosya Yapısı
* `DataCoSupplyChainDataset.csv`: Ham veri seti (Kaggle).
* `logistic_analysis.ipynb`: Veri temizleme, simülasyon ve ML kodları.
* `dashboard_hazir_veri.csv`: BI araçları için temizlenmiş metrikler.

## 📈 Sonuç ve Öneriler
1. **Yüksek Riskli Şehirler:** Belirli lokasyonlardaki lojistik ağının yeniden optimize edilmesi gerekmektedir.
2. **Hızlı Geçiş (Fast Track):** Modelin "Gecikme Riski" verdiği yüksek tutarlı siparişlerin depoda önceliklendirilmesi önerilir."""

with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

print("README.md başarıyla oluşturuldu!")